### Script purpose: Ming office title coding

1. General principles:
    - A comprehensive ontological structure of office title includes four parts: `Classification + Administrative Unit (optional) + Function (optional) + Title`
    - Each part corresponds to a table.
    - Separate `coding_value` and `raw_value`.
        - `raw_value`: the string appeared in original book text.
        - `coding_value`: the revised string that can be successfully coded.

2. Notes:
    - `Office title by LENGTH` table merges CBDB Ming office title with UCI table. Duplicates in CBDB table are removed in this table, i.e., this is the clean table we are going to use.

### TODO:
- [×] English words;
- [×] Forward slash.

In [1]:
% matplotlib inline
import sqlite3
import pandas as pd
import networkx as nx
import xlrd
import matplotlib.pyplot as plt
import math
import warnings
from tqdm import tqdm
import re
warnings.filterwarnings('ignore')
plt.style.use('ggplot')

### `c_office_chn` from UCI.

In [2]:
df_uci_office_ming=pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSCmhbCk1B-9jjINMhy_VwikM6_Sn7bjdO7b_vaZJkVcYCCYlWVlhYVCFtAs0fPX-UEO62GWxaX1qAS/pub?gid=630627340&single=true&output=tsv',
                                    sep='\t')
df_uci_office_ming=df_uci_office_ming[['c_office_id（Dictionary Ser#)','Institution 1', 'Institution 2', 'Institution 3', 'c_office_chn']].rename(columns={'c_office_id（Dictionary Ser#)':'c_office_id'})
df_uci_office_ming['c_office_chn']=[s.replace('/', '') for s in df_uci_office_ming['c_office_chn']]
df_uci_office_ming.sample(3)

,c_office_id,Institution 1,Institution 2,Institution 3,c_office_chn
3982,70764,皇族宮廷類 Imperial Family and Royal Court,宦官門 Eunuch Offices,御前近侍 Emporor's Eunuch Attendants,乾清宮管事
816,1596,中央輔佐官署類 Central Administration Assistance,寺監門 Courts and Directorates,太僕寺 The Court of the Imperial Stud,牧監監副
4009,621,皇族宮廷類 Imperial Family and Royal Court,宦官門 Eunuch Offices,御用監（洪武舊制）The Directorate for Imperial Accouter...,令


In [3]:
df_uci_office_ming['inst_1_chn']=[str(s).split()[0].replace('nan', '') for s in df_uci_office_ming['Institution 1']]
df_uci_office_ming['inst_2_chn']=[str(s).split()[0].replace('nan', '') for s in df_uci_office_ming['Institution 2']]
df_uci_office_ming['inst_3_chn']=[str(s).split()[0].replace('nan', '') for s in df_uci_office_ming['Institution 3']]
df_uci_office_ming['uci_value']=df_uci_office_ming['inst_1_chn']+df_uci_office_ming['inst_2_chn']+df_uci_office_ming['inst_3_chn']+df_uci_office_ming['c_office_chn']
df_uci_office_ming['c_office_id']=pd.to_numeric(df_uci_office_ming['c_office_id'], errors='coerce')
df_uci_office_ming.drop(['inst_1_chn', 'inst_2_chn', 'inst_3_chn', 'Institution 1', 'Institution 2', 'Institution 3', 'c_office_chn'], axis=1, inplace=True)

In [4]:
df_uci_office_ming[df_uci_office_ming['c_office_id'].duplicated()]

,c_office_id,uci_value
1130,71508.0,中央輔佐官署類秘書門翰林院直文淵閣侍講學士
1195,71503.0,中央輔佐官署類考官門會試官知貢舉官
1219,72165.0,中央輔佐官署類考官門鄉試官順天同考官
1282,NaN,京衛京營與中央軍事官署類京營門京營京營總兵官
2314,71504.0,地方官署類省官門行中書省理問所知事
2718,71274.0,地方軍事與治安機構類招討經略安撫使門宣撫司宣撫司經歷
2821,NaN,文武散階勛爵類勛爵門伯平涼伯
2842,NaN,文武散階勛爵類勛爵門伯新城伯
2862,NaN,文武散階勛爵類勛爵門伯永定伯
2882,NaN,文武散階勛爵類勛爵門伯鎮遠伯


In [5]:
df_uci_office_ming['uci_value']=[s.replace('/', '') for s in df_uci_office_ming['uci_value']]
df_uci_office_ming['uci_value']=[s.replace('／', '') for s in df_uci_office_ming['uci_value']]
df_uci_office_ming['uci_value']=[s.replace('、', '') for s in df_uci_office_ming['uci_value']]
df_uci_office_ming['uci_value']=[re.sub(r'[a-zA-Z]', string=s, repl='') for s in df_uci_office_ming['uci_value']]

In [6]:
df_uci_office_ming.drop(df_uci_office_ming[df_uci_office_ming['c_office_id'].duplicated()].index, inplace=True)
df_uci_office_ming.set_index('c_office_id', inplace=True)
df_uci_office_ming.sample(3)

,uci_value
c_office_id,
659.0,皇族宮廷類宦官門兵仗司（洪武舊制）鋒利奉御
143.0,皇族宮廷類宗室門王府長史司直史
71723.0,文武散階勛爵類勛爵門侯昌寧侯


### `c_office_chn` from CBDB uncleaned, and merge with UCI.

In [7]:
conn = sqlite3.connect('../../SQL/sqlite_20180302.db')
df_cbdb_office_ming=pd.read_sql_query("SELECT * FROM OFFICE_CODES", conn)[pd.read_sql_query("SELECT * FROM OFFICE_CODES", conn).c_dy==19].set_index('c_office_id')
df_cbdb_office_ming.sample(3)

,tts_sysno,c_dy,c_office_pinyin,c_office_chn,c_office_pinyin_alt,c_office_chn_alt,c_office_trans,c_office_trans_alt,c_source,c_pages,c_notes,c_category_1,c_category_2,c_category_3,c_category_4,c_office_id_old
c_office_id,,,,,,,,,,,,,,,,
70545,15282.0,19,jian guo,監國,None,None,Regent,None,NaN,None,None,Regent,None,None,None,0.0
70559,15296.0,19,jiang du guan,講讀官,None,None,Instructional Officials,None,NaN,None,None,Instructional Officials,None,None,None,0.0
71165,15902.0,19,xing bu zuo shi lang,行部左侍郎,None,None,Acting Left Vice Minister of a Ministry,None,NaN,None,None,Acting Left Vice Minister of a Ministry,None,None,None,0.0


In [8]:
for index in tqdm(df_uci_office_ming.index):
    if index in df_cbdb_office_ming.index:
        df_uci_office_ming.loc[index, 'cbdb_value']=df_cbdb_office_ming.loc[index, 'c_office_chn']
        df_uci_office_ming.loc[index, 'tts_sysno']=df_cbdb_office_ming.loc[index, 'tts_sysno']
        df_uci_office_ming.loc[index, 'c_office_pinyin']=df_cbdb_office_ming.loc[index, 'c_office_pinyin']
        df_uci_office_ming.loc[index, 'c_office_pinyin_alt']=df_cbdb_office_ming.loc[index, 'c_office_pinyin_alt']
        df_uci_office_ming.loc[index, 'c_office_chn_alt']=df_cbdb_office_ming.loc[index, 'c_office_chn_alt']
        df_uci_office_ming.loc[index, 'c_office_trans']=df_cbdb_office_ming.loc[index, 'c_office_trans']
        df_uci_office_ming.loc[index, 'c_office_trans_alt']=df_cbdb_office_ming.loc[index, 'c_office_trans_alt']
        df_uci_office_ming.loc[index, 'c_source']=df_cbdb_office_ming.loc[index, 'c_source']
        df_uci_office_ming.loc[index, 'c_pages']=df_cbdb_office_ming.loc[index, 'c_pages']
        df_uci_office_ming.loc[index, 'c_notes']=df_cbdb_office_ming.loc[index, 'c_notes']
        df_uci_office_ming.loc[index, 'c_category_1']=df_cbdb_office_ming.loc[index, 'c_category_1']
        df_uci_office_ming.loc[index, 'c_category_2']=df_cbdb_office_ming.loc[index, 'c_category_2']
        df_uci_office_ming.loc[index, 'c_category_3']=df_cbdb_office_ming.loc[index, 'c_category_3']
        df_uci_office_ming.loc[index, 'c_category_4']=df_cbdb_office_ming.loc[index, 'c_category_4']
        df_uci_office_ming.loc[index, 'c_office_id_old']=df_cbdb_office_ming.loc[index, 'c_office_id_old']
df_uci_office_ming.loc[index, 'c_dy']=19

100%|██████████| 4304/4304 [00:31<00:00, 137.70it/s]


In [9]:
df_office_ming_merged=df_uci_office_ming
df_office_ming_merged.sample(3)

,uci_value,cbdb_value,tts_sysno,c_office_pinyin,c_office_pinyin_alt,c_office_chn_alt,c_office_trans,c_office_trans_alt,c_source,c_pages,c_notes,c_category_1,c_category_2,c_category_3,c_category_4,c_office_id_old,c_dy
c_office_id,,,,,,,,,,,,,,,,,
2604.0,地方官署類京府門應天府稅課局大使,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004.0,京衛京營與中央軍事官署類五城兵馬指揮司門西城兵馬指揮司吏目,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70307.0,地方官署類府官門府官推官,府推官,15044.0,fu tui guan,* tui guan;,*推官;,Judge in a Prefecture,None,NaN,NaN,NaN,Judge in a Prefecture,NaN,NaN,NaN,0.0,NaN


### Coding `c_office_chn`.

### TODO: - Done.
    - [×] Subtract titles from right.
    - [×] Add appointment type.
    - [×] Use online revised CLS table.

In [10]:
df_adm=pd.read_csv('../data_output/C_OT_ADM.tsv', sep='\t').set_index('c_ot_adm_id')
df_cls=pd.read_csv('../data_output/C_OT_CLS.tsv', sep='\t').set_index('c_ot_cls_id')
df_tit=pd.read_csv('../data_output/C_OT_TIT.tsv', sep='\t').set_index('c_ot_tit_id')
df_func=pd.read_csv('../data_output/C_OT_FUNC.tsv', sep='\t').set_index('c_ot_func_id')
df_app_ty=pd.read_csv('../data_output/APPOINTMENT_TYPE_CODES.tsv', sep='\t').set_index('c_appt_type_code')

In [11]:
df_tit.sample(3)

,c_ot_tit_chinm,value_to_run,c_ot_tit_desc,c_ot_tit_start,c_ot_tit_end,length
c_ot_tit_id,,,,,,
430,東莞侯,2.0,NaN,NaN,NaN,3
86,都督同知,1.0,NaN,NaN,NaN,4
440,伏羌侯,2.0,NaN,NaN,NaN,3


In [12]:
df_office_ming_merged['c_ot_coding']=df_office_ming_merged['uci_value']

In [13]:
# Replace titles (only one title in an office title string).
for ming_ot_index in tqdm(df_office_ming_merged.index):
    ming_ot = df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']
    ming_ot_done=[]
    for tit_index in df_tit.index:
        tit=df_tit.loc[tit_index, 'c_ot_tit_chinm']
        if ming_ot.endswith(tit) and ming_ot not in ming_ot_done:
            df_office_ming_merged.loc[ming_ot_index, 'c_ot_tit_chinm']=tit
            df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']=re.findall(r'(.+)'+tit, ming_ot)[0]+'T'+str(tit_index)
            ming_ot_done.append(ming_ot)
df_office_ming_merged.sample(3)

100%|██████████| 4304/4304 [02:44<00:00, 26.16it/s]


,uci_value,cbdb_value,tts_sysno,c_office_pinyin,c_office_pinyin_alt,c_office_chn_alt,c_office_trans,c_office_trans_alt,c_source,c_pages,c_notes,c_category_1,c_category_2,c_category_3,c_category_4,c_office_id_old,c_dy,c_ot_coding,c_ot_tit_chinm
c_office_id,,,,,,,,,,,,,,,,,,,
3146.0,文武散階勛爵類文勛階門資治尹,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,文武散階勛爵類文勛階門T671,資治尹
70968.0,中央輔佐官署類秘書門通政使司經歷司知事,通政司事,15705.0,tong zheng si shi,None,None,[Not Yet Translated],NaN,NaN,NaN,NaN,[Not Yet Translated],NaN,NaN,NaN,0.0,NaN,中央輔佐官署類秘書門通政使司經歷司T808,知事
1310.0,中央中樞官署類六部門工部總部員外郎,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,中央中樞官署類六部門工部總部T632,員外郎


In [14]:
# Replace Classifications (can be multiple units in an office title string).
for ming_ot_index in tqdm(df_office_ming_merged.index):
    cls_list=[]
    for cls_index in df_cls.index:
        cls=df_cls.loc[cls_index, 'c_ot_cls_chinm']
        c_ot_coding = df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']
        if cls in c_ot_coding:
            cls_list.append(cls)
            df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']=c_ot_coding.replace(cls, 'C'+str(cls_index))
    if cls_list!=[]:
        df_office_ming_merged.loc[ming_ot_index, 'c_ot_cls_chinm']='#'.join(cls_list)
df_office_ming_merged.sample(3)

100%|██████████| 4304/4304 [00:34<00:00, 123.50it/s]


,uci_value,cbdb_value,tts_sysno,c_office_pinyin,c_office_pinyin_alt,c_office_chn_alt,c_office_trans,c_office_trans_alt,c_source,c_pages,c_notes,c_category_1,c_category_2,c_category_3,c_category_4,c_office_id_old,c_dy,c_ot_coding,c_ot_tit_chinm,c_ot_cls_chinm
c_office_id,,,,,,,,,,,,,,,,,,,,
70443.0,京衛京營與中央軍事官署類大都督府門後軍都督府左都督,後軍左都督,15180.0,hou jun zuo du du,None,None,Left Commissioner-in-chief of the Chief Milita...,None,NaN,NaN,NaN,Left Commissioner-in-chief of the Chief Milita...,NaN,NaN,NaN,0.0,NaN,C0C114後軍都督府T486,左都督,京衛京營與中央軍事官署類#大都督府門
1145.0,中央中樞官署類六部門兵部右侍中,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C11C45兵部T670,右侍中,中央中樞官署類#六部門
72735.0,司法監察機構類監察門總督巡撫官總督漕運,總理河道漕運,17463.0,zong li he dao cao yun,None,None,None,None,NaN,NaN,NaN,None,NaN,NaN,NaN,0.0,NaN,C7C58C113T1636,總督漕運,司法監察機構類#總督巡撫官#監察門


In [15]:
# Replace admin units (can be multiple units in an office title string).
for ming_ot_index in tqdm(df_office_ming_merged.index):
    adm_list=[]
    for adm_index in df_adm.index:
        adm=df_adm.loc[adm_index, 'c_ot_adm_chinm']
        c_ot_coding = df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']
        if adm in c_ot_coding:
            adm_list.append(adm)
            df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']=c_ot_coding.replace(adm, 'A'+str(adm_index))
    if adm_list!=[]:
        df_office_ming_merged.loc[ming_ot_index, 'c_ot_adm_chinm']='#'.join(adm_list)
df_office_ming_merged.sample(3)

100%|██████████| 4304/4304 [02:05<00:00, 34.31it/s]


,uci_value,cbdb_value,tts_sysno,c_office_pinyin,c_office_pinyin_alt,c_office_chn_alt,c_office_trans,c_office_trans_alt,c_source,c_pages,...,c_category_1,c_category_2,c_category_3,c_category_4,c_office_id_old,c_dy,c_ot_coding,c_ot_tit_chinm,c_ot_cls_chinm,c_ot_adm_chinm
c_office_id,,,,,,,,,,,,,,,,,,,,,
2354.0,南京官署類南京寺監門南京太常寺祠祭署祀丞,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,C26C19A1143A315T1111,祀丞,南京官署類#南京寺監門,南京太常寺#祠祭署
3151.0,文武散階勛爵類文勛階門協正庶尹,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,C6C36T1513,協正庶尹,文武散階勛爵類#文勛階門,NaN
2532.0,牧鹽舶政類馬政門苑馬寺寺丞,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,C21C42A1056T793,寺丞,牧鹽舶政類#馬政門,苑馬寺


In [16]:
# Replace functional units (can be multiple units in an office title string).
for ming_ot_index in tqdm(df_office_ming_merged.index):
    func_list=[]
    for func_index in df_func.index:
        func=df_func.loc[func_index, 'c_ot_func_chinm']
        c_ot_coding = df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']
        if func in c_ot_coding:
            func_list.append(func)
            df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']=c_ot_coding.replace(func, 'F'+str(func_index))
    if func_list!=[]:
        df_office_ming_merged.loc[ming_ot_index, 'c_ot_func_chinm']='#'.join(func_list)
df_office_ming_merged.sample(3)

100%|██████████| 4304/4304 [00:16<00:00, 265.98it/s]


,uci_value,cbdb_value,tts_sysno,c_office_pinyin,c_office_pinyin_alt,c_office_chn_alt,c_office_trans,c_office_trans_alt,c_source,c_pages,...,c_category_2,c_category_3,c_category_4,c_office_id_old,c_dy,c_ot_coding,c_ot_tit_chinm,c_ot_cls_chinm,c_ot_adm_chinm,c_ot_func_chinm
c_office_id,,,,,,,,,,,,,,,,,,,,,
71625.0,京衛京營與中央軍事官署類京衛門京衛總旗,總旗,16362.0,zong qi,None,None,Plattoon Commander,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,C0C43A1191T1695,總旗,京衛京營與中央軍事官署類#京衛門,京衛,NaN
71014.0,京衛京營與中央軍事官署類京衛門親軍指揮使司指揮使,衛親軍指揮使,15751.0,wei qin jun zhi hui shi,None,None,Commander of the Imperial Armies in a Guard,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,C0C43A22T161,指揮使,京衛京營與中央軍事官署類#京衛門,親軍指揮使司,NaN
1359.0,中央中樞官署類六部門行部兵曹清吏司主事,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,C11C45A970A111T1294,主事,中央中樞官署類#六部門,兵曹清吏司#行部,NaN


In [17]:
# Replace appointment type.
for ming_ot_index in tqdm(df_office_ming_merged.index):
    app_ty_list=[]
    for app_ty_index in df_app_ty.index:
        app_ty=df_app_ty.loc[app_ty_index, 'c_appt_type_desc_chn']
        c_ot_coding = df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']
        if app_ty in c_ot_coding:
            app_ty_list.append(app_ty)
            df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']=c_ot_coding.replace(app_ty, 'P'+str(app_ty_index))
    if app_ty_list!=[]:
        df_office_ming_merged.loc[ming_ot_index, 'c_ot_app_chinm']='#'.join(app_ty_list)
df_office_ming_merged.sample(3)

100%|██████████| 4304/4304 [00:17<00:00, 250.16it/s]


,uci_value,cbdb_value,tts_sysno,c_office_pinyin,c_office_pinyin_alt,c_office_chn_alt,c_office_trans,c_office_trans_alt,c_source,c_pages,...,c_category_3,c_category_4,c_office_id_old,c_dy,c_ot_coding,c_ot_tit_chinm,c_ot_cls_chinm,c_ot_adm_chinm,c_ot_func_chinm,c_ot_app_chinm
c_office_id,,,,,,,,,,,,,,,,,,,,,
71561.0,地方軍事與治安機構類(行)都指揮使司門衛指揮使司中衛指揮使,中衛指揮使,16298.0,zhong wei zhi hui shi,None,None,Commander of the Middle Guard,None,NaN,NaN,...,NaN,NaN,0.0,NaN,C1C2A1060中衛T161,指揮使,地方軍事與治安機構類#(行)都指揮使司門,衛指揮使司,NaN,NaN
72047.0,皇族宮廷類帝后門內命婦信王妃,信王妃,16784.0,xin wang fei,None,None,[Not Yet Translated],爵,NaN,NaN,...,NaN,NaN,0.0,NaN,C23C39C64T656,信王妃,皇族宮廷類#內命婦#帝后門,NaN,NaN,NaN
1956.0,京衛京營與中央軍事官署類大都督府門右軍都督府右斷事,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,C0C114A1094T446,右斷事,京衛京營與中央軍事官署類#大都督府門,右軍都督府,NaN,NaN


In [18]:
for index in tqdm(df_office_ming_merged.index):
    c_ot_coding=df_office_ming_merged.loc[index, 'c_ot_coding']
    if re.sub(r'A|C|T|F|P|\d', '', string=c_ot_coding)!='':
        df_office_ming_merged.loc[index, 'pass']='F'
    else:
        df_office_ming_merged.loc[index, 'pass']='T'

100%|██████████| 4304/4304 [00:04<00:00, 1040.91it/s]


In [19]:
df_office_ming_merged_coded=pd.read_excel('https://docs.google.com/spreadsheets/d/e/2PACX-1vQwXjRmlMR9w2ZV2tcenPSz9UgE7WAgeumGxxCJlceQOZRQFgm6_mgMCAlC_GzM0yxxNsDOlU1-5aH-/pub?output=xlsx',
                                          sheetname='merged_tbl_coding'
                                         )
df_office_ming_merged_coded.set_index('c_office_id', inplace=True)
df_office_ming_merged_coded.sample(3)

,uci_value,cbdb_value,tts_sysno,c_office_pinyin,c_office_pinyin_alt,c_office_chn_alt,c_office_trans,c_office_trans_alt,c_source,c_pages,...,c_office_id_old,c_dy,c_ot_coding,c_ot_tit_chinm,c_ot_cls_chinm,c_ot_adm_chinm,c_ot_func_chinm,c_ot_app_chinm,pass,type
c_office_id,,,,,,,,,,,,,,,,,,,,,
757.0,皇族宮廷類女官門尚宮局司簿司掌簿,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,C23C37A1017A310T1035,掌簿,皇族宮廷類#女官門,尚宮局#司簿司,NaN,NaN,T,NaN
2647.0,地方官署類省官門行中書省左司員外郎,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,C17C54A1062A1197T632,員外郎,地方官署類#省官門,行中書省#左司,NaN,NaN,T,NaN
72022.0,文武散階勛爵類勛爵門公夏國公,夏國公,16759.0,xia guo gong,NaN,NaN,[Not Yet Translated],爵,NaN,NaN,...,0.0,NaN,C6C41C70T423,夏國公,文武散階勛爵類#勛爵門#公,NaN,NaN,NaN,T,NaN


In [20]:
for c_office_id in df_office_ming_merged.index:
    df_office_ming_merged.loc[c_office_id, 'type']=df_office_ming_merged_coded.loc[c_office_id, 'type']

In [21]:
df_office_ming_merged.to_excel('../data_output/ming_office_title_merged_coding.xlsx', encoding='utf8')